<a href="https://colab.research.google.com/github/marcoullmann/enron/blob/master/enron_graph_analysis_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preprocessing for network graph analysis
This Notebook will transform the Enron email dataset and the calculated topic vector to files, which can be used for Netowrk Graph analysis.

In [1]:
import glob
import pandas as pd
import numpy as np
import pickle

from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/enron/"

num_topics=32

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/enron


In [0]:
def get_unique_email(df):
  from_unique = df.From.unique()
  to_unique = df.To.unique()
  all_unique =np.unique([*from_unique, *to_unique])

  print("Unique From addresses: " + str(len(from_unique)))
  print("Unique To addresses: " + str(len(to_unique)))
  print("Unique overall addresses: " + str(len(all_unique)))
  return all_unique

## Create vertices
Load the eMails and create a row for each interaction between two email addresses.

In [0]:
files = glob.glob("./data/emails_df_*.pkl")
dfs = []
for file in files:
  df = pd.read_pickle(file)
  df.drop(df.columns.difference(['Message-ID', 'From', 'To']), 1, inplace=True)
  dfs.append(df)
emails_df = pd.concat(dfs)
emails_df = emails_df.dropna()

In [0]:
emails_df['From'] = emails_df['From'].apply(lambda x:next(iter(x)))

In [0]:
to = emails_df.apply(lambda x: pd.Series(list(x['To'])),axis=1).stack().reset_index(level=1, drop=True)
to.name = 'To'
emails_df = emails_df.drop('To', axis=1).join(to)

In [0]:
fromto_df = emails_df.reset_index()

In [0]:
# Call this instead of execute the above code (faster)
fromto_df = pickle.load( open("./data/from_to_links.pkl", "rb") )

One single email can now be on multiple rows:

In [5]:
fromto_df.loc[fromto_df['Message-ID'] == '<17564024.1075847132119.JavaMail.evans@thyme>']

,Message-ID,From,To
774143,<17564024.1075847132119.JavaMail.evans@thyme>,lola.willis@enron.com,thomas.engel@enron.com
774144,<17564024.1075847132119.JavaMail.evans@thyme>,lola.willis@enron.com,connie.kwan@enron.com
774145,<17564024.1075847132119.JavaMail.evans@thyme>,lola.willis@enron.com,mary.gosnell@enron.com
774146,<17564024.1075847132119.JavaMail.evans@thyme>,lola.willis@enron.com,jennifer.denny@enron.com
774147,<17564024.1075847132119.JavaMail.evans@thyme>,lola.willis@enron.com,francesc.guillen@enron.com
774148,<17564024.1075847132119.JavaMail.evans@thyme>,lola.willis@enron.com,judy.thorne@enron.com
774149,<17564024.1075847132119.JavaMail.evans@thyme>,lola.willis@enron.com,mark.taylor@enron.com
774150,<17564024.1075847132119.JavaMail.evans@thyme>,lola.willis@enron.com,lisa.lees@enron.com
774151,<17564024.1075847132119.JavaMail.evans@thyme>,lola.willis@enron.com,stephanie.sever@enron.com
774152,<17564024.1075847132119.JavaMail.evans@thyme>,lola.willis@enron.com,tom.moran@enron.com


Some data cleansing. Remove rows with invalid email addresses.

In [6]:
email_pattern = "(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)"
valid_from_email_filter = fromto_df['From'].str.contains(email_pattern) 
valid_to_email_filter = fromto_df['To'].str.contains(email_pattern)

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [0]:
fromto_df = fromto_df[valid_from_email_filter & valid_to_email_filter]

In [8]:
vertices = pd.DataFrame(get_unique_email(fromto_df))
vertices.columns = ['id']

Unique From addresses: 19529
Unique To addresses: 65911
Unique overall addresses: 74988


In [0]:
# pickle.dump(fromto_df, open('./data/from_to_links.pkl', 'wb'))
vertices.columns = vertices.columns.astype(str)
vertices.to_parquet("./data/vertices.parquet")

### Filter out all external people and people with less than 200 interactions in general and less than 5 interactions between two people.




In [10]:
print("fromto_df.shape: " + str(fromto_df.shape))

from_enron_filter = fromto_df['From'].str.contains("@enron.com")
to_enron_filter = fromto_df['To'].str.contains("@enron.com")
same_address_filter = fromto_df.From == fromto_df.To

fromto_enron_df = fromto_df[from_enron_filter & to_enron_filter & ~same_address_filter]
print("fromto_enron_df.shape: " + str(fromto_enron_df.shape))
get_unique_email(fromto_enron_df)

fromto_df.shape: (3055344, 3)
fromto_enron_df.shape: (2435508, 3)
Unique From addresses: 6228
Unique To addresses: 18339
Unique overall addresses: 19035


array(['.brady@enron.com', '.dow@enron.com', '.eber@enron.com', ...,
       'zuzana.strmenova@enron.com', 'zyhorie.allyson@enron.com',
       'zykorie.allyson@enron.com'], dtype='<U49')

In [11]:
fromto_enron_from_df = fromto_enron_df.groupby('From').agg('count')['Message-ID'].to_frame().reset_index().sort_values(by='Message-ID', ascending=False)
fromto_enron_to_df = fromto_enron_df.groupby('To').agg('count')['Message-ID'].to_frame().reset_index().sort_values(by='Message-ID', ascending=False)

from_unique = fromto_enron_from_df[fromto_enron_from_df['Message-ID']>200].From.unique()
to_unique = fromto_enron_to_df[fromto_enron_to_df['Message-ID']>200].To.unique()
all_unique =np.unique([*from_unique, *to_unique])

print("Unique From addresses: " + str(len(from_unique)))
print("Unique To addresses: " + str(len(to_unique)))
print("Unique overall addresses: " + str(len(all_unique)))

Unique From addresses: 1054
Unique To addresses: 2266
Unique overall addresses: 2605


In [0]:
vertices_small = pd.DataFrame(all_unique)
vertices_small.columns = ['id']

In [0]:
# pickle.dump(vertices_small, open('./data/vertices_small.pkl', 'wb'))
vertices_small.columns = vertices_small.columns.astype(str)
vertices_small.to_parquet("./data/vertices_small.parquet")

#Create edges

In [14]:
files = glob.glob("./data/emails_df_*.pkl")
dfs = []
for file in files:
  df = pd.read_pickle(file)
  df.drop(df.columns.difference(['Message-ID','Date']), 1, inplace=True)
  dfs.append(df)
emails_df = pd.concat(dfs)
print("emails_df.shape: " + str(emails_df.shape))

emails_df.shape: (517401, 1)


In [15]:
topic_vector = pickle.load( open("./model/topic_vector.pkl", "rb") )
topic_vector_df = pd.DataFrame(topic_vector, columns=range(num_topics)).applymap(lambda x: x[1])
print("topic_vector_df.shape: " + str(topic_vector_df.shape))

topic_vector_df.shape: (517401, 32)


In [0]:
emails_topic_vector_df = pd.concat([emails_df.reset_index(), topic_vector_df.reset_index(drop=True)], axis=1)

In [0]:
edges = pd.merge(fromto_df, emails_topic_vector_df, how='left', on='Message-ID')
edges.columns = edges.columns.astype(str)
edges.to_parquet("./data/edges.parquet")

###Filter out all external people and people with less than 200 interactions.

In [0]:
edges_small = pd.merge(fromto_enron_df, emails_topic_vector_df, how='left', on='Message-ID')
edges_small.columns = edges_small.columns.astype(str)
edges_small.to_parquet("./data/edges_small.parquet")

In [19]:
fields = {'Message-ID':'count'}
for i in range(0,num_topics):
  fields[str(i)] = 'sum' 
print("edges_small.shape: " + str(edges_small.shape))
edges_small_agg = edges_small.groupby(['From','To']).agg(fields).sort_values('Message-ID').reset_index()
print("edges_small_agg.shape: " + str(edges_small_agg.shape))

edges_small.shape: (2435508, 36)
edges_small_agg.shape: (194042, 35)


In [0]:
edges_small_agg = edges_small_agg[edges_small_agg['Message-ID']>3]

In [0]:
edges_small_agg.to_parquet("./data/edges_small_agg.parquet")

###Vertices with aggregated topics

In [0]:
from_agg_df = edges_small.groupby(['From']).agg(fields).sort_values('Message-ID').reset_index()
#from_agg_df.drop('Message-ID', inplace=True)
from_agg_df.rename(columns={"From": "id"}, inplace = True)

to_agg_df = edges_small.groupby(['To']).agg(fields).sort_values('Message-ID').reset_index()
#to_agg_df.drop('Message-ID', inplace=True)
to_agg_df.rename(columns={"To": "id"}, inplace=True)

# Assumption: Weight of received emails is 0.5 compared to sent emails
ranger = [str(x) for x in range(0,num_topics)]
to_agg_df[ranger] = to_agg_df[ranger].apply(lambda x: x/2)

In [0]:
vertices_small_agg = pd.concat([from_agg_df,to_agg_df]).groupby(['id']).agg(fields).sort_values('Message-ID').reset_index()

In [24]:
vertices_small_agg = vertices_small_agg[vertices_small_agg.id.isin(vertices_small.id)]
vertices_small_agg.drop('Message-ID', inplace=True)

KeyError: ignored

In [0]:
vertices_small_agg.to_parquet("./data/vertices_small_agg.parquet")